In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
from custom_dataset import Binary_Dataset
from matplotlib import pyplot as plt
from PIL import Image
import os
from inception import inception_v3
from Trainer import train_loop
from torch.optim import Adam
%matplotlib inline
torch.cuda.empty_cache()

# Define Dataloaders for each class


In [ ]:
#define relevant paths
labels_dir="Data\list_attr_celeba.csv"
image_dir="Data\img_align_celeba\img_align_celeba"

############################################define batch size for all data loaders here
bathsize=32

##############################################define num workers for data laoders here
workers=4

#define composed transforms
composed = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([299,299]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])


labels=["Male","Smiling","Wavy_Hair"]
label_number=2
#Choose which label 
ds=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Train")
dataset_loader=DataLoader(ds,batch_size=bathsize, shuffle=True, num_workers=workers)

ds_val=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Val")
val_loader=DataLoader(ds_val,batch_size=bathsize, shuffle=True, num_workers=workers)

"""
# male
ds1=Binary_Dataset(labels_dir, labels[0],image_dir ,transform=composed)
dataset_loader=DataLoader(ds1,batch_size=bathsize, shuffle=True, num_workers=workers)

# Smiling
ds2=Binary_Dataset(labels_dir, labels[1],image_dir ,transform=composed)
dataset_loader=DataLoader(ds2,batch_size=bathsize, shuffle=True, num_workers=workers)

# wavy hair
ds3=Binary_Dataset(labels_dir, labels[2],image_dir ,transform=composed)
dataset_loader=DataLoader(ds3,batch_size=bathsize, shuffle=True, num_workers=workers)

# ???????????????
ds4=Binary_Dataset(labels_dir, labels[3],image_dir ,transform=composed)
dataset_loader=DataLoader(ds4,batch_size=bathsize, shuffle=True, num_workers=workers)
"""

"""
for idx, (data, image) in enumerate(ds):
    print(idx)
"""


data=next(iter(dataset_loader))
sample, target = data
#print(sample)
print(target)


In [ ]:
"""
#wont work anymore because pil
idx=0
plt.imshow(sample[idx])
print(labels[label_number])
print(target[idx])
"""

# Define CNN's

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=inception_v3(pretrained=False).to(device)

# Training Loop

In [ ]:
learning_rate=0.0001
save_name="Wavy_Hair_12_9_5epochs_extractionline"
optimizer = Adam(params=model.parameters(), lr=learning_rate)
train_loop(model=model,dataloader=dataset_loader,epochs=5, optimizer=optimizer,save_name="Wavy_Hair_12_8_5epochs_extractionline")

In [ ]:
#check validation accuracy
ds_val=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Val")
val_loader=DataLoader(ds_val,batch_size=bathsize, shuffle=True, num_workers=workers)
device = 'cuda'
val_data=iter(val_loader)
#model=model.to(device)

model.eval()

correct=0
tested_samples=0
limit=2000
for data in val_data:
    sample, target = data
    
    #send to device
    sample=sample.to(device)
    target=target.to(device)

    pred=torch.argmax(model(sample),dim=1,keepdim=True)

    correct+=torch.eq(pred,target.view_as(pred)).sum()
    tested_samples+=pred.size()[0]
    if tested_samples>limit:
        break

    print(correct)

#print precision
print(correct.item()/tested_samples)

In [ ]:
torch.save(model.state_dict(),save_name+".pt")  

In [ ]:
#check test accuracy

#def test(epoch)
ds_val=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Test")
val_loader=DataLoader(ds_val,batch_size=bathsize, shuffle=True, num_workers=workers)
device = 'cuda'
val_data=iter(val_loader)
#model=model.to(device)

model.eval()

correct=0
tested_samples=0
limit=2000
for data in val_data:
    sample, target = data
    
    #send to device
    sample=sample.to(device)
    target=target.to(device)

    pred=torch.argmax(model(sample),dim=1,keepdim=True)

    correct+=torch.eq(pred,target.view_as(pred)).sum()
    tested_samples+=pred.size()[0]
    if tested_samples>limit:
        break

    print(correct)

#print precision
print(correct.item()/tested_samples)

In [ ]:
# # test

# #total_epoch = 1 ###################################################### number of epochs over here

# bs = 10
# import warnings
# warnings.filterwarnings('ignore')

# epoch = 1
# r_list = []
# p_list = []
# acc_list = []
# AUC_list = []
# # TP = 0
# # TN = 0
# # FN = 0
# # FP = 0
# vote_pred = np.zeros(testset.__len__())
# vote_score = np.zeros(testset.__len__())


# targetlist, scorelist, predlist = test(epoch)
# print('target',targetlist)
# print('score',scorelist)
# print('predict',predlist)
# vote_pred = vote_pred + predlist 
# vote_score = vote_score + scorelist 

# TP = ((predlist == 1) & (targetlist == 1)).sum()

# TN = ((predlist == 0) & (targetlist == 0)).sum()
# FN = ((predlist == 0) & (targetlist == 1)).sum()
# FP = ((predlist == 1) & (targetlist == 0)).sum()

# print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
# print('TP+FP',TP+FP)
# p = TP / (TP + FP)
# print('precision',p)
# p = TP / (TP + FP)
# r = TP / (TP + FN)
# print('recall',r)
# F1 = 2 * r * p / (r + p)
# acc = (TP + TN) / (TP + TN + FP + FN)
# print('F1',F1)
# print('acc',acc)
# AUC = roc_auc_score(targetlist, vote_score)
# print('AUC', AUC)

In [ ]:
#check feature extrcation

# 162770
# 19867
# 19962

bathsize = 19867
FE_ds=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Val")
FE_loader=DataLoader(FE_ds,batch_size=bathsize, shuffle=False, num_workers=workers)

device = 'cuda'

FE_data=iter(FE_loader)

#model=model.to(device)

model.eval()

correct=0
tested_samples=0
limit=10


l = ['label']
range_int = range(1,2049)
r = [str(x) for x in range_int]
col = l + r   
px = pd.DataFrame(columns = col)  
px.to_csv('___________img.csv', mode='a', index=False, header=True)
    
    
for data in FE_data:
    sample, target = data
    
    #send to device
    sample=sample.to(device)
    target=target.to(device)
    
    output, extraction = model(sample)
        
    target_val = target.cpu().numpy()
    label_val = target_val[0]
        
    extract = extraction.cpu().numpy()
            
    data = np.insert(extract, 0, label_val, axis = None)
    data = np.reshape(data, (1, 2049))
 
    px = pd.DataFrame(data)   
    px.to_csv('__________img.csv', mode='a', index=False, header=False)    
        
    
    pred=torch.argmax(output,dim=1,keepdim=True)

    correct+=torch.eq(pred,target.view_as(pred)).sum()
    tested_samples+=pred.size()[0]
    if tested_samples>limit:
        break

    print(correct)

#print precision
print(correct.item()/tested_samples)